# Run all files

## Generation of accumulated point cloud

In [1]:
"""
Load dataset
"""

import os
import time

start_time = time.time()

data_path = os.path.abspath("parsed_kitti_data_181212.pickle")


import pickle

with open(data_path, 'rb') as f:
    parsed_kitti_data = pickle.load(f)

print "Dataset was loaded"

"""
Data structure
"""

for date in parsed_kitti_data:
    print "date:", date
    
    for drive in parsed_kitti_data[date]:
        print "\tdrive:", drive

Dataset was loaded
date: 2011_09_26
	drive: 0048
	drive: 0005
	drive: 0059
	drive: 0014
	drive: 0013
	drive: 0002
	drive: 0011
	drive: 0017
	drive: 0051
	drive: 0057
	drive: 0056
	drive: 0009
	drive: 0060
	drive: 0084
	drive: 0018
	drive: 0093
	drive: 0091
	drive: 0001


In [2]:
import copy
import pickle


parsed_kitti_data_w_r = {}

for date in parsed_kitti_data:
    print "date:", date
    parsed_kitti_data_w_r[date] = {}
    
    for drive in parsed_kitti_data[date]:
                
        print "\tdrive:", drive
        
        parsed_kitti_data_w_r_drive = copy.deepcopy(parsed_kitti_data_w_r)
        
        parsed_kitti_data_w_r_drive[date][drive] = []
        
        for tracklets_list in parsed_kitti_data[date][drive]:
            
            tracklets_list_w_r = {}
            
            print "\t\ttrack: ",
            for key in tracklets_list:
                
                print " ", key,
                tracklets_list_w_r[key] = tracklets_list[key]
                
            print "{}".format(len(tracklets_list[key]))
            
            parsed_kitti_data_w_r_drive[date][drive].append(tracklets_list_w_r)
            
        """ 
        Save pickle
        """
        out_file_nme = "parsed_kitti_data_181212_{}.pickle".format(drive)
        with open(out_file_nme, 'wb') as f:
            pickle.dump(parsed_kitti_data_w_r_drive, f, protocol=2)

        print("\n\nSaved new dataset, protocol=2, {}\n\n".format(out_file_nme))
        del parsed_kitti_data_w_r_drive
            

print "\n\n======================================================="
print "======================================================="        
print "Execution time: {} ".format( time.time() - start_time )
print "======================================================="
print "=======================================================\n\n"

date: 2011_09_26
	drive: 0048
		track:    type   trans   rot   rect   pc 6
		track:    type   trans   rot   rect   pc 6
		track:    type   trans   rot   rect   pc 3
		track:    type   trans   rot   rect   pc 22
		track:    type   trans   rot   rect   pc 22
		track:    type   trans   rot   rect   pc 19
		track:    type   trans   rot   rect   pc 19
		track:    type   trans   rot   rect   pc 18


Saved new dataset, protocol=2, parsed_kitti_data_181212_0048.pickle


	drive: 0005
		track:    type   trans   rot   rect   pc 154
		track:    type   trans   rot   rect   pc 154
		track:    type   trans   rot   rect   pc 6
		track:    type   trans   rot   rect   pc 103
		track:    type   trans   rot   rect   pc 9
		track:    type   trans   rot   rect   pc 35
		track:    type   trans   rot   rect   pc 33
		track:    type   trans   rot   rect   pc 33
		track:    type   trans   rot   rect   pc 28
		track:    type   trans   rot   rect   pc 34
		track:    type   trans   rot   rect   pc 30
		track:    t



Saved new dataset, protocol=2, parsed_kitti_data_181212_0051.pickle


	drive: 0057
		track:    type   trans   rot   rect   pc 361
		track:    type   trans   rot   rect   pc 24
		track:    type   trans   rot   rect   pc 323
		track:    type   trans   rot   rect   pc 18
		track:    type   trans   rot   rect   pc 32
		track:    type   trans   rot   rect   pc 48
		track:    type   trans   rot   rect   pc 57
		track:    type   trans   rot   rect   pc 319
		track:    type   trans   rot   rect   pc 332
		track:    type   trans   rot   rect   pc 262
		track:    type   trans   rot   rect   pc 35
		track:    type   trans   rot   rect   pc 22
		track:    type   trans   rot   rect   pc 38
		track:    type   trans   rot   rect   pc 43
		track:    type   trans   rot   rect   pc 39
		track:    type   trans   rot   rect   pc 40
		track:    type   trans   rot   rect   pc 26
		track:    type   trans   rot   rect   pc 33
		track:    type   trans   rot   rect   pc 27
		track:    type   trans   rot   rec



Saved new dataset, protocol=2, parsed_kitti_data_181212_0084.pickle


	drive: 0018
		track:    type   trans   rot   rect   pc 8
		track:    type   trans   rot   rect   pc 43
		track:    type   trans   rot   rect   pc 46
		track:    type   trans   rot   rect   pc 45
		track:    type   trans   rot   rect   pc 44
		track:    type   trans   rot   rect   pc 49
		track:    type   trans   rot   rect   pc 45
		track:    type   trans   rot   rect   pc 37
		track:    type   trans   rot   rect   pc 36
		track:    type   trans   rot   rect   pc 70
		track:    type   trans   rot   rect   pc 52
		track:    type   trans   rot   rect   pc 56
		track:    type   trans   rot   rect   pc 136
		track:    type   trans   rot   rect   pc 55
		track:    type   trans   rot   rect   pc 29


Saved new dataset, protocol=2, parsed_kitti_data_181212_0018.pickle


	drive: 0093
		track:    type   trans   rot   rect   pc 10
		track:    type   trans   rot   rect   pc 28
		track:    type   trans   rot   rect   pc 53
		

## Registration

In [3]:
import re

"""
Search all files 
"""
basedir = './'
p = re.compile('parsed_kitti_data_181212_\d{4}')

drives = []

cnt = 0
for root,d_names,f_names in os.walk(basedir):
    for f in f_names:
        m = p.match(f)
        if m:
            cnt +=1
            print 'Match found: ', m.group(), cnt
            drives.append(m.group() + ".pickle")

Match found:  parsed_kitti_data_181212_0048 1
Match found:  parsed_kitti_data_181212_0013 2
Match found:  parsed_kitti_data_181212_0091 3
Match found:  parsed_kitti_data_181212_0060 4
Match found:  parsed_kitti_data_181212_0057 5
Match found:  parsed_kitti_data_181212_0002 6
Match found:  parsed_kitti_data_181212_0005 7
Match found:  parsed_kitti_data_181212_0011 8
Match found:  parsed_kitti_data_181212_0056 9
Match found:  parsed_kitti_data_181212_0059 10
Match found:  parsed_kitti_data_181212_0084 11
Match found:  parsed_kitti_data_181212_0093 12
Match found:  parsed_kitti_data_181212_0018 13
Match found:  parsed_kitti_data_181212_0017 14
Match found:  parsed_kitti_data_181212_0014 15
Match found:  parsed_kitti_data_181212_0001 16
Match found:  parsed_kitti_data_181212_0009 17
Match found:  parsed_kitti_data_181212_0051 18


In [4]:
import tqdm
import sys
import pickle


from registration import *


for d in drives:
    
    """
    Load dataset
    """
    with open(d, 'rb') as f:
        parsed_kitti_data_w_r = pickle.load(f)
    

    for date in parsed_kitti_data_w_r:
        print "date:", date
        
        assert len(parsed_kitti_data_w_r[date]) == 1

        for drive in parsed_kitti_data_w_r[date]:

            print "\tdrive:", drive

            sys.stdout.flush()

            for i in tqdm.trange( len(parsed_kitti_data_w_r[date][drive]) ):

                pc = parsed_kitti_data_w_r[date][drive][i]['pc']

                """
                Registration
                """
                parsed_kitti_data_w_r[date][drive][i]['pc_wr'] = []
                sys.stdout.flush()
                if len(pc) > 1:
                    parsed_kitti_data_w_r[date][drive][i]['pc_wr'] = registration_layered(pc)
                else:
                    parsed_kitti_data_w_r[date][drive][i]['pc_wr'] = []

                    print "\tdrive:", drive
                    
    """
    Save data
    """
    out_data_path = 'parsed_kitti_data_w_r_181212_{}.pickle'.format(drive)
    
    with open(out_data_path, 'wb') as f:
        pickle.dump(parsed_kitti_data_w_r, f, protocol=2)
    
    print("\n\nSaved new dataset, protocol=2")
    
print "\n\n======================================================="
print "======================================================="        
print "Execution time: {} ".format( time.time() - start_time )
print "======================================================="
print "=======================================================\n\n"

date: 2011_09_26
	drive: 0048


100%|██████████| 8/8 [01:17<00:00,  9.74s/it]



Saved new dataset, protocol=2
date: 2011_09_26
	drive: 0013



  0%|          | 0/9 [00:00<?, ?it/s]/home/gozilla/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/gozilla/.local/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 9/9 [05:54<00:00, 39.36s/it] 




Saved new dataset, protocol=2
date: 2011_09_26
	drive: 0091


...



Saved new dataset, protocol=2
date: 2011_09_26
	drive: 0009


100%|██████████| 98/98 [03:52<00:00,  2.37s/it]




Saved new dataset, protocol=2
date: 2011_09_26
	drive: 0051


100%|██████████| 50/50 [19:52<00:00, 23.84s/it]




Saved new dataset, protocol=2


Execution time: 11185.1096821 




## Classification

In [5]:
import re

"""
Search all files 
"""
basedir = './'
p = re.compile('parsed_kitti_data_w_r_181212_\d{4}')

drives = []

cnt = 0
for root,d_names,f_names in os.walk(basedir):
    for f in f_names:
        m = p.match(f)
        if m:
            cnt +=1
            print 'Match found: ', m.group(), cnt
            drives.append(m.group() + ".pickle")

Match found:  parsed_kitti_data_w_r_181212_0056 1
Match found:  parsed_kitti_data_w_r_181212_0014 2
Match found:  parsed_kitti_data_w_r_181212_0013 3
Match found:  parsed_kitti_data_w_r_181212_0059 4
Match found:  parsed_kitti_data_w_r_181212_0093 5
Match found:  parsed_kitti_data_w_r_181212_0018 6
Match found:  parsed_kitti_data_w_r_181212_0057 7
Match found:  parsed_kitti_data_w_r_181212_0091 8
Match found:  parsed_kitti_data_w_r_181212_0011 9
Match found:  parsed_kitti_data_w_r_181212_0051 10
Match found:  parsed_kitti_data_w_r_181212_0060 11
Match found:  parsed_kitti_data_w_r_181212_0005 12
Match found:  parsed_kitti_data_w_r_181212_0017 13
Match found:  parsed_kitti_data_w_r_181212_0009 14
Match found:  parsed_kitti_data_w_r_181212_0001 15
Match found:  parsed_kitti_data_w_r_181212_0002 16
Match found:  parsed_kitti_data_w_r_181212_0048 17
Match found:  parsed_kitti_data_w_r_181212_0084 18


In [6]:
import tqdm
import sys
import pickle
import os, sys
import tensorflow as tf

from registration import *

""" 
Import model 
"""
sys.path.append( os.path.abspath('../../Network/model') )
from train import *
import model_out_5_heading_3_relu as MODEL
model_path = os.path.abspath('../../Network/notebook/train_log_20181208/model_out_1/model009.ckpt')

def NormalizeResample(data, num_sample):
    """ data is in N x ...
    we want to keep num_samplexC of them.
    if N > num_sample, we will randomly keep num_sample of them.
    if N < num_sample, we will randomly duplicate samples.
    """
  

    ## normalizing    
    min_val = np.min(data, axis = 0)
    
    data[:,0] = data[:,0] - min_val[0]
    data[:,1] = data[:,1] - min_val[1]
    data[:,2] = data[:,2] - min_val[2]
      
    max_val = np.max(data)
    
    data[:,0] = data[:,0] / max_val
    data[:,1] = data[:,1] / max_val
    data[:,2] = data[:,2] / max_val
                 
        
    ## resampling
    N = data.shape[0]
    if (N == num_sample):
        return data
    elif (N > num_sample):
        sample = np.random.choice(N, num_sample)
        return data[sample, ...]
    else:
        sample = np.random.choice(N, num_sample-N)
        dup_data = data[sample, ...]
        return np.concatenate([data, dup_data], 0)

    
    
# for d in drives:
    
def run_classification(d):
    """
    Load dataset
    """
    with open(d, 'rb') as f:
        parsed_kitti_data_w_r = pickle.load(f)

        """
        Classification
        """

        is_training = False

        GPU_INDEX = 0
        NUM_POINT = 1024
        NUM_CLASSES = 3
        BATCH_SIZE = 1

        # reset graph
        tf.reset_default_graph()

        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl, heading_labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=())

            # simple model
            pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)

            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()

        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = True

        ops = {'pointclouds_pl': pointclouds_pl,
               'is_training_pl': is_training_pl,
               'pred': pred}

        # ops.reset_default_graph() 

        sess = tf.Session(config=config)

        # Restore variables from disk.
        saver.restore(sess, model_path)

        print "Model restored"
        sys.stdout.flush()


        for date in parsed_kitti_data_w_r:
            print "date:", date

            for drive in parsed_kitti_data_w_r[date]:
                print "\tdrive:", drive

                for i in range( len(parsed_kitti_data_w_r[date][drive]) ):

                    clusters = parsed_kitti_data_w_r[date][drive][i]['pc_wr']
                    parsed_kitti_data_w_r[date][drive][i]['cls_wr'] = []
                    for j in tqdm.trange(len(clusters)):

                        if len(clusters[j]) == 0:
                            continue

                        current_data = NormalizeResample(clusters[j], NUM_POINT)
                        feed_dict = {ops['pointclouds_pl']: current_data.reshape((BATCH_SIZE, NUM_POINT, 3)),
                                     ops['is_training_pl']: is_training}
                        pred_val = sess.run([ops['pred']], feed_dict=feed_dict )

                        cls = np.argmax(pred_val[0][:3], 1)
                        parsed_kitti_data_w_r[date][drive][i]['cls_wr'].append(cls)

                    clusters = parsed_kitti_data_w_r[date][drive][i]['pc']
                    parsed_kitti_data_w_r[date][drive][i]['cls'] = []
                    for j in tqdm.trange(len(clusters)):

                        if len(clusters[j]) == 0:
                            continue
                        current_data = NormalizeResample(clusters[j][:,:3], NUM_POINT)
                        feed_dict = {ops['pointclouds_pl']: current_data.reshape((BATCH_SIZE, NUM_POINT, 3)),
                                     ops['is_training_pl']: is_training}
                        pred_val = sess.run([ops['pred']], feed_dict=feed_dict )

                        cls = np.argmax(pred_val[0][:3], 1)
                        parsed_kitti_data_w_r[date][drive][i]['cls'].append(cls)
                    
    """
    Save data
    """
    out_data_path = 'parsed_kitti_data_w_r_c_181212_{}.pickle'.format(drive)
    
    with open(out_data_path, 'wb') as f:
        pickle.dump(parsed_kitti_data_w_r, f, protocol=2)
    
    print("\n\nSaved new dataset, protocol=2")
    
    


In [7]:
import multiprocessing

# option 1: execute code with extra process

## arguments: out_path, trained_model = None, hlw = 0.1, max_epoch = 1):

for d in drives:
    p = multiprocessing.Process( target=run_classification, args = (d,) )

    p.start()
    p.join()

INFO:tensorflow:Restoring parameters from /home/gozilla/kyungpyo/git/MasterThesisProject/Network/notebook/train_log_20181208/model_out_1/model009.ckpt
Model restored
date: 2011_09_26
	drive: 0056


100%|██████████| 25/25 [00:00<00:00, 79.83it/s]
0it [00:00, ?it/s]
100%|██████████| 18/18 [00:00<00:00, 84.65it/s]




Saved new dataset, protocol=2
INFO:tensorflow:Restoring parameters from /home/gozilla/kyungpyo/git/MasterThesisProject/Network/notebook/train_log_20181208/model_out_1/model009.ckpt
Model restored
date: 2011_09_26
	drive: 0014


100%|██████████| 10/10 [00:00<00:00, 99.58it/s]
0it [00:00, ?it/s]
100%|██████████| 12/12 [00:00<00:00, 74.01it/s] 




Saved new dataset, protocol=2
INFO:tensorflow:Restoring parameters from /home/gozilla/kyungpyo/git/MasterThesisProject/Network/notebook/train_log_20181208/model_out_1/model009.ckpt
Model restored
date: 2011_09_26
	drive: 0013


...



Saved new dataset, protocol=2
INFO:tensorflow:Restoring parameters from /home/gozilla/kyungpyo/git/MasterThesisProject/Network/notebook/train_log_20181208/model_out_1/model009.ckpt
Model restored
date: 2011_09_26
	drive: 0048


100%|██████████| 18/18 [00:00<00:00, 110.84it/s]




Saved new dataset, protocol=2
INFO:tensorflow:Restoring parameters from /home/gozilla/kyungpyo/git/MasterThesisProject/Network/notebook/train_log_20181208/model_out_1/model009.ckpt
Model restored
date: 2011_09_26
	drive: 0084


100%|██████████| 21/21 [00:00<00:00, 98.98it/s] 




Saved new dataset, protocol=2


In [8]:
print "\n\n======================================================="
print "======================================================="        
print "Execution time: {} ".format( time.time() - start_time )
print "======================================================="
print "=======================================================\n\n"



Execution time: 11613.483418 


